# TDA@YSDA
## Seminar 3

In [ ]:
!pip install --upgrade Ripser
!pip install --upgrade diagram2vec

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

from ripser import lower_star_img
from ripser import Rips
vr = Rips()

import persim
import diagram2vec

from scipy.ndimage import gaussian_filter

from sklearn.datasets import make_circles
from sklearn.manifold import MDS, SpectralEmbedding

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import Dataset
torch.set_default_dtype(torch.float64)

import pickle
import h5py
import math
from tqdm import tqdm

## Persistent homology, persistent diagrams, Wasserstein distance and stability

In [ ]:
# original data
X, y = make_circles(n_samples=200, noise=0.1)
X = X[y==0]

Topology studies data invariant to continous transformations, so topological invariants like (persistent) homology will not change under such class of transformations.

**Exercise**

Apply rotation and dilation transformations to copy of original data.

In [ ]:
theta = np.radians(30)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c,-s), (s, c)))
print(R)

In [ ]:
# transformed data
X_transformed = np.copy(X)
X_transformed[:,0] = X[:,0] * 0.75
X_transformed = np.dot(X_transformed, R)

In [ ]:
fig = plt.figure(figsize=(8,4))

plt.subplot(121)
plt.title("Data")
plt.grid(linestyle="dotted")
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.scatter(X[:,0], X[:,1], alpha=0.33)

plt.subplot(122)
plt.title("Transformed data")
plt.grid(linestyle="dotted")
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.scatter(X_transformed[:,0], X_transformed[:,1], alpha=0.33)

plt.show()

**Exercise**

Compute persistence diagrams of a filtration of Vietoris-Rips complex built on point cloud data

In [ ]:
diagram = vr.fit_transform(X)
diagram_transformed = vr.fit_transform(X_transformed)

In [ ]:
diagram

In [ ]:
fig = plt.figure(figsize=(8,4))
plt.suptitle("Persistent diagram of a filtration")

plt.grid(linestyle="dotted")

plt.subplot(121)
plt.title("Data")
plt.grid(linestyle="dotted")
vr.plot(diagram)

plt.subplot(122)
plt.title("Transformed data")
plt.grid(linestyle="dotted")
vr.plot(diagram_transformed)

plt.show()

One can define the geometry on the space of persistent diagrams, defining a metric on it. Optimal transport approach is used to compare persistent diagrams which are multisets of intervals of arbitrary cardinality.  

The variants of optimal transport distances are _Wasserstein-2 distance_, and its approximations like _sliced Wasserstein distance_ and _Bollteneck distance_, which is Wasserstein-$\infty$ distance.

**Exercise**

Compute Bottleneck `persim.bottleneck` and sliced Wasserstein distances `persim.sliced_wasserstein` between perisistent diagrams of original and transformed data.

In [ ]:
diagram

In [ ]:
diagram[1].shape

In [ ]:
diagram_transformed[1].shape

In [ ]:
persim.bottleneck(diagram[1], diagram_transformed[1])

In [ ]:
persim.sliced_wasserstein(diagram_transformed[1], diagram_transformed[1])

Bottlneck distance used a single matching between most discriminative pair of points.

**Exercise**

Visualize Bottleneck matching.

In [ ]:
# compute Bottleneck distance matching
d, matching = persim.bottleneck(diagram[1], diagram_transformed[1], matching=True)

In [ ]:
# plot Bottleneck distance matchign
fig = plt.figure(figsize=(8,4))
plt.suptitle("Bottleneck distance matching")
persim.bottleneck_matching(diagram[1], diagram_transformed[1], matching, labels=['Original $H_1$', 'Transformed $H_1$'])


For Bottleneck distance stability to small perturbations is theoretically proved.

**Exercise**

Plot Bottleneck distance with respect to different level of Gaussian noise applied to original data.

In [ ]:
# original data
X_orig, y = make_circles(n_samples=200, noise=0.0)
X_orig = X_orig[y==0]
diagram_orig = vr.fit_transform(X_orig)

# your code here
dists = []

for noise in tqdm(np.arange(0, 0.26, 0.02)):
    # your code here
    X_noisy, _ = make_circles(n_samples=500, noise=noise)
    X_noisy = X_noisy
    
    diagram_noise = vr.fit_transform(X_noisy)
    
    dists.append(persim.bottleneck(diagram_orig[1], diagram_noise[1]))

In [ ]:
dists

In [ ]:
plt.plot(dists)

## 3. Persistent homology of graphs

Pipeline is as follows:

1. compute persistent diagrams via Ripser 
2. compute vectorization of diagrams, so-called persistent images and Betti curves
3. apply classifier on vectorization

In [ ]:
# load data
X_graphs = pickle.load(open("./data/metric_graphs/X.pkl", "rb"))
y = pickle.load(open("./data/metric_graphs/y_all.pkl", "rb"))
y_dnod = pickle.load(open("./data/metric_graphs/y_d_nod.pkl", "rb"))

y_col = ["a"] * len(y)
y_col = np.array(y_col)

y_col[y==0] = "blue"
y_col[y==2] = "green"
y_col[y==1] = "red"
y_col[y==3] = "yellow"

### Compute persistent diagrams

In [ ]:
# add h_1 diagrams only
maxdim = 1
h = 1

rips = Rips(maxdim=maxdim)

diagrams = []
for x in X_graphs:
    diagrams.append(rips.fit_transform(x, distance_matrix=True)[h])

len(diagrams)

### Clusterization

In [ ]:
%%time
n = len(X_graphs)
distances = np.zeros((n, n))

for i in range(0, n):
    for j in range(i+1, n):
        distances[i,j] = persim.sliced_wasserstein(diagrams[i], diagrams[j])
        
distances_symmetrize = distances + distances.T

In [ ]:
distances_symmetrize = distances + distances.T

In [ ]:
mds = MDS(n_components=10, max_iter=3000, eps=1e-9, dissimilarity="precomputed", random_state=1, n_jobs=-1)
X_metric = mds.fit(distances_symmetrize).embedding_

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")

plt.scatter(X_metric[:, 0], X_metric[:, 1], c=y_col)
plt.show()

### Vectorization

Persistent diagram is a multiset of intervals of arbitrary length which is can not be handled by methods of machine learning.

#### Persistent images

One possible to solutions besides providing a metric on the space of persistent diagrams is vectorization of diagrams to a vector of fixed length.

In [ ]:
pi = persim.PersImage(spread=0.025, pixels=[32, 32])
pimages = np.array(pi.transform(diagrams))

pimages.shape

In [ ]:
plt.figure(figsize=(8,4))

plt.suptitle("$H_" + str(h) + "$ diagram")

plt.subplot(121)
plt.title("Persistent diagram")
rips.plot(diagrams[0], legend=False)

plt.subplot(122)
plt.title("Persistent image")
pi.show(pimages[0])

plt.show()

### Classification

In [ ]:
X_all = pimages.reshape((pimages.shape[0], -1))
y_all = pickle.load(open("./data/metric_graphs/y_all.pkl", "rb")).astype(int)

X_control = X_all[y_all==0]
X_depression = X_all[y_all==1]
X = np.concatenate((X_control, X_depression), axis=0)
y = np.concatenate((np.zeros(25), np.ones(25)), axis=0)

In [ ]:
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

accuracies = []

for train_index, test_index in skf.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    model = LogisticRegression(penalty='l2', C=10.0, solver='liblinear', random_state=42)

    model.fit(X_train, y_train)
    accuracies.append(model.score(X_test, y_test))

print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracies), np.std(accuracies)))

#### Betti curves

In [ ]:
X_betti_curve = diagram2vec.persistence_curve(diagrams, m=20)
X_betti_curve.shape

In [ ]:
X_betti_curve[0,1]

In [ ]:
plt.figure(figsize=(5,5))
ax = plt.gca()
plt.xlim(0,1)
plt.title("Betti curve", pad=15)
plt.xlabel("t", fontsize=12, ha="right", x=1)
plt.ylabel("", ha="right", y=1)
plt.grid(linestyle="dotted")

plt.step(np.linspace(0,1,20), X_betti_curve[0,0], color="b", where="post", linewidth=2, label="Pers-1")
plt.step(np.linspace(0,1,20), X_betti_curve[0,1], color="r", where="post", linewidth=2, label="Pers-2")
plt.legend(fontsize=12)

plt.show()

In [ ]:
X_control = X_betti_curve[0][y_all==0]
X_depression = X_betti_curve[0][y_all==1]
X = np.concatenate((X_control, X_depression), axis=0)
y = np.concatenate((np.zeros(25), np.ones(25)), axis=0)

In [ ]:
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

accuracies = []

for train_index, test_index in skf.split(X, y):
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]

    model = RandomForestClassifier(random_state=42)

    model.fit(X_train, y_train)
    accuracies.append(model.score(X_test, y_test))

print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracies), np.std(accuracies)))

## 4. Persistent homology of digital images

Persistence Diagrams with Linear Machine Learning Models (Obayashi, Hiraoka), 2017  
https://arxiv.org/abs/1706.10082

In [ ]:
W = 300
sigma1 = 4
sigma2 = 2
t = 0.01

def generate(N, S, W=300, sigma1=4, sigma2=2, t=0.01, bins=64):

    z = np.zeros((N, S, 2))
    for n in range(N):
        z[n, 0] = np.random.uniform(0, W, size=(2))
        for s in range(S-1):
            d_1 = np.random.normal(0, sigma1)
            d_2 = np.random.normal(0, sigma1)
            z[n, s+1, 0] = (z[n, s, 0] + d_1) % W
            z[n, s+1, 1] = (z[n, s, 1] + d_2) % W

    z_r = z.reshape(N*S, 2)
    H, _, _ = np.histogram2d(z_r[:,0], z_r[:,1], bins=bins)
    
    G = gaussian_filter(H, sigma2)
    G[G < t] = 0
    
    return G

### Image generation

Generate 100 images accoring to model A and model B

In [ ]:
images = np.zeros((100,64,64))

# class A
N = 100
S = 30

for n in range(50):
    images[n] = generate(N, S)
    
# class B
N = 250
S = 10

for n in range(50):
    images[n+50] = generate(N, S)

In [ ]:
fig = plt.figure()
plt.gray()

ax1 = fig.add_subplot(121)
plt.title("Class A")

ax2 = fig.add_subplot(122)
plt.title("Class B")

ax1.imshow(images[int(np.random.uniform(0, 50))])
ax2.imshow(images[int(np.random.uniform(51, 100))])

plt.show()

### Compute persistent diagrams

In [ ]:
diags = []

for i in range(images.shape[0]):
    diags.append(lower_star_img(images[i])[:-1])

In [ ]:
plt.figure(figsize=(4,4))
persim.plot_diagrams(diags[52])

### Vectorization

In [ ]:
pi = persim.PersImage(spread=0.025, pixels=[32, 32], verbose=False)
pers_images = np.array(pi.transform(diags))

betti_curves = diagram2vec.persistence_curve(diags, m=25)

pers_images.shape, betti_curves.shape

### Classification

#### Persistent images

In [ ]:
X_images = images.reshape((pers_images.shape[0], -1))
y = np.concatenate((np.zeros(50), np.ones(50)), axis=0)

y_col = ["b"] * len(y)
y_col = np.array(y_col)

y_col[y==1] = "r"

In [ ]:
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

accuracies = []

for train_index, test_index in skf.split(X_images, y):
    X_train, y_train = X_images[train_index], y[train_index]
    X_test, y_test = X_images[test_index], y[test_index]

    model = RandomForestClassifier(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    accuracies.append(model.score(X_test, y_test))

print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracies), np.std(accuracies)))

#### Betti curves

In [ ]:
X_betti_curves = betti_curves[0]
y = np.concatenate((np.zeros(50), np.ones(50)), axis=0)

y_col = ["b"] * len(y)
y_col = np.array(y_col)

y_col[y==1] = "r"

In [ ]:
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

accuracies = []

for train_index, test_index in skf.split(X_betti_curves, y):
    X_train, y_train = X_betti_curves[train_index], y[train_index]
    X_test, y_test = X_betti_curves[test_index], y[test_index]

    model = RandomForestClassifier(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)
    accuracies.append(model.score(X_test, y_test))

print("Accuracy: {:.4f} ± {:.4f}".format(np.mean(accuracies), np.std(accuracies)))

### Clusterization

**Exercise**

Compute the two-dimensional embeddings using linear and nonlinear techniques learned during the course, given persistent images, Betti curves and pairwise distances between data points.

In [ ]:
X_emb = SpectralEmbedding().fit_transform(X_betti_curves)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")

plt.scatter(X_emb[:, 0], X_emb[:, 1], c=y_col)
plt.show()

### Vectorization with neural networks

#### Deep sets

Zaheer et al. _Deep sets (2017)_

Given a set $\{x_i\}_{i=1}^n \in (\mathbb{R}^d)^n$ get an embedding $\mathrm{vec} \in \mathbb{R}^D$ as

$$
\begin{equation}
\mathrm{vec}(\{x_i\}_{i=1}^n) = \rho \left( \sum_{i=1}^n \phi(x_i) \right),
\end{equation}
$$

where $\phi: \mathbb{R}^d \rightarrow \mathbb{R}^{d'}$ is an _encoder_ and $\rho: \mathbb{R}^{d'} \rightarrow \mathbb{R}^D$ is a _decoder_ mappings given by neural networks. Sum plays a role of permutation invariant pooling, and can be replaced with any other permutation invariant one, such as mean or max pooling.

#### Perslay

Carrière et al. _Perslay: A neural network layer for persistence diagrams (2020)_

Perslay provides specific encoder/decoder/pooling triples which output Betti curves, persistent images or landscapes.

- Betti curve: 
- persistent image: 
- persistent landscape: 

#### Persformer
Reinauer et al. _Persformer: A Transformer Architecture for Topological Machine Learning (2021)_

Persformer sets encoder $\phi$ with a self-attention map instead of multi-layer perceptron used in Deep sets.

In [ ]:
class DeepSets(torch.nn.Module):
    
    def __init__(self, n_items):
        super(DeepSets, self).__init__()
        self.enc = Encoder(3, 12, 24, n_items)
        self.dec = Decoder(24, 12, 2)
        
    def forward(self, X):
        return self.dec(self.enc(X))
    
class Encoder(torch.nn.Module):
    
    def __init__(self, n_in, n_hidden, n_out, n_items):
        super(Encoder, self).__init__()
        self.n_items = n_items
        self.linear1 = SetLinear(n_in, n_hidden, n_items)
        self.linear2 = SetLinear(n_hidden, n_out, n_items)
        
    def forward(self, X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X)).sum(axis=1, keepdim=True).reshape((self.n_items, -1))
        return X
    
class Decoder(torch.nn.Module):
    
    def __init__(self, n_in, n_hidden, n_out):
        super(Decoder, self).__init__()
        self.linear1 = torch.nn.Linear(n_in, n_hidden)
        self.linear2 = torch.nn.Linear(n_hidden, n_out)
        
    def forward(self, X):
        X = F.relu(self.linear1(X))
        X = self.linear2(X)
        return X

class SetLinear(torch.nn.Module):
    
    def __init__(self, n_in, n_out, n_items): 
        super(SetLinear, self).__init__()
        
        W = torch.zeros((n_out, n_in))
        init.kaiming_uniform_(W, a=math.sqrt(5))
        W_block_diag = torch.block_diag(*W.unsqueeze(0).repeat(n_items,1,1))
        self.weight = torch.nn.Parameter(W_block_diag)
        
        self.register_parameter("bias", None)
        
    def forward(self, input):
        return self.weight @ input

In [ ]:
def filter_topk(diagram, topk0, topk1):
    
    idx0, idx1 = diagram[:,0] == 0., diagram[:,0] == 1.
    
    idx0_sorted = (diagram[idx0][:,2] - diagram[idx0][:,1]).argsort(descending=True)
    diagram0 = diagram[idx0][idx0_sorted][:topk0]
    
    idx1_sorted = (diagram[idx1][:,2] - diagram[idx1][:,1]).argsort(descending=True)
    diagram1 = diagram[idx1][idx1_sorted][:topk1]
    
    return np.vstack((diagram0, diagram1))

In [ ]:
class ObayashiDataset(Dataset):
    
    def __init__(self, filename, topk0=35, topk1=65, transform=None):
        self.f = h5py.File(filename, "r")
        self.topk0 = topk0
        self.topk1 = topk1
    
    def __len__(self):
        return len(self.f["/target"])
    
    def __getitem__(self, idx):
        feature_idx = np.array([0,1,2])
        #X = filter_topk(torch.tensor(self.f["/diagram/" + str(idx)][:,feature_idx]), self.topk0, self.topk1)
        X = torch.tensor(self.f["/diagram/" + str(idx)][:,feature_idx])
        y = torch.tensor(self.f["/target"][idx])
        return X, y

In [ ]:
def dataloader(dataset, batchsize=50, shuffle=False):
    
    if shuffle:
        idx = np.arange(len(dataset))
        np.random.shuffle(idx)
    
    # for each minibatch
    for start_idx in range(0, len(dataset) - batchsize + 1, batchsize):
        
        if shuffle:
            batch_idx = idx[start_idx:start_idx + batchsize]
        else:
            batch_idx = slice(start_idx, start_idx + batchsize)
    
        # for each index
        Xs = []
        ys = []
        ks = []
        n = 0
        
        for item_idx in batch_idx:
            X, y = dataset[item_idx]
            Xs.append(X.T)
            ys.append(y)
            ks.append(X.shape[0])
            d = X.shape[1]
            
        batch = np.zeros((batchsize * d, sum(ks)))
        y = np.array(ys)
        
        for i, X in enumerate(Xs):
            start_i = i*d
            start_j = sum(ks[:i])
            
            end_i = start_i + d
            end_j = start_j + ks[i]
            
            batch[start_i:end_i,start_j:end_j] = X
            
        yield torch.tensor(batch), torch.tensor(y)
    

#### Train loop

In [ ]:
dataset = ObayashiDataset("./data/obayashi.hdf5")

In [ ]:
epochs = 50
batchsize = 25

model = DeepSets(n_items=batchsize)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
accs = []

for t in range(epochs):

    size = len(dataset)
    
    losses = []
    accuracies = []
    
    for batch, (X, y) in enumerate(dataloader(dataset, batchsize=batchsize, shuffle=True)):    
        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #if batch % 1 == 10:
        loss, current = loss.item(), batch * len(y)
        accuracy = ((F.softmax(pred, dim=1).argmax(dim=1) == y).type(torch.float).sum() / batchsize).item()
        
        losses.append(loss)
        accuracies.append(accuracy)
        accs.append(np.mean(accuracies))
        
    print("Epoch: {:<2}, loss: {:.4f}, acc: {:.4f}".format(t+1, np.mean(losses), np.mean(accuracies)))